In [3]:
import csv
import requests
from bs4 import BeautifulSoup

# Carregar arquivo CSV
csv_file = '/mnt/data/apartamentos.csv'

# Links para pesquisa
links = [
    "https://www.quintoandar.com.br/imovel/894601086/alugar/apartamento-2-quartos-santa-ifigenia-sao-paulo?from_route=%22house%20details%22&search_id=%229c5941646d2911ef9c4d8e0e165fee2f%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%7D",
    "https://www.quintoandar.com.br/imovel/894590276/alugar/apartamento-3-quartos-bela-vista-sao-paulo",
    "https://www.quintoandar.com.br/imovel/892877492/alugar/apartamento-3-quartos-bela-vista-sao-paulo"
]

# Função para coletar dados de um link
def coletar_dados(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Função auxiliar para evitar erros de 'NoneType'
    def get_text_or_none(element):
        return element.get_text(strip=True) if element else "Não informado"
    
    # Coletar informações do imóvel
    nome_endereco = get_text_or_none(soup.select_one("h1.address-title"))
    bairro = get_text_or_none(soup.select_one("span.neighborhood"))
    preco_aluguel = get_text_or_none(soup.select_one("span.rental-price"))
    condominio = get_text_or_none(soup.select_one("span.condominium-fee"))
    iptu = get_text_or_none(soup.select_one("span.iptu-price"))
    quartos = get_text_or_none(soup.select_one("span.bedroom-count"))
    banheiros = get_text_or_none(soup.select_one("span.bathroom-count"))
    vagas = get_text_or_none(soup.select_one("span.parking-space-count"))
    area = get_text_or_none(soup.select_one("span.area-size"))
    facilidades = ", ".join([f.get_text(strip=True) for f in soup.select("span.facility-name")]) if soup.select("span.facility-name") else "Não informado"
    nome_imobiliaria = "QuintoAndar"  # Ajustar para coletar nome da imobiliária caso necessário
    site = link
    
    # Calcular o valor total mensal (caso um dos valores não seja encontrado, ele será tratado)
    try:
        valor_total_mensal = float(preco_aluguel.replace("R$", "").replace(",", "").strip()) + \
                             float(condominio.replace("R$", "").replace(",", "").strip()) + \
                             float(iptu.replace("R$", "").replace(",", "").strip())
    except ValueError:
        valor_total_mensal = "Não informado"

    # Retornar os dados coletados
    return {
        "nome_endereco": nome_endereco,
        "bairro": bairro,
        "preco_aluguel": preco_aluguel,
        "condominio": condominio,
        "iptu": iptu,
        "quartos": quartos,
        "banheiros": banheiros,
        "vagas": vagas,
        "area": area,
        "facilidades": facilidades,
        "nome_imobiliaria": nome_imobiliaria,
        "site": site,
        "valor_total_mensal": valor_total_mensal
    }

# Função para inserir dados no CSV
def inserir_no_csv(dados):
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Inserir linha no CSV
        writer.writerow([
            dados['nome_endereco'],
            dados['bairro'],
            dados['preco_aluguel'],
            dados['condominio'],
            dados['iptu'],
            dados['quartos'],
            dados['banheiros'],
            dados['vagas'],
            dados['area'],
            dados['facilidades'],
            dados['nome_imobiliaria'],
            dados['site'],
            dados['valor_total_mensal']
        ])

# Coletar e exibir os dados para todos os links
for link in links:
    dados_imovel = coletar_dados(link)
    print(f"Dados coletados do link: {link}")
    for chave, valor in dados_imovel.items():
        print(f"{chave}: {valor}")
    
    # Solicitar confirmação para inserir no CSV
    confirmacao = input("Deseja inserir os dados deste imóvel no CSV? (sim/não): ").strip().lower()
    if confirmacao == "sim":
        inserir_no_csv(dados_imovel)
        print(f"Dados do imóvel inseridos no CSV com sucesso.")
    else:
        print(f"Dados do imóvel não foram inseridos no CSV.")

Dados coletados do link: https://www.quintoandar.com.br/imovel/894601086/alugar/apartamento-2-quartos-santa-ifigenia-sao-paulo?from_route=%22house%20details%22&search_id=%229c5941646d2911ef9c4d8e0e165fee2f%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%7D
nome_endereco: Não informado
bairro: Não informado
preco_aluguel: Não informado
condominio: Não informado
iptu: Não informado
quartos: Não informado
banheiros: Não informado
vagas: Não informado
area: Não informado
facilidades: Não informado
nome_imobiliaria: QuintoAndar
site: https://www.quintoandar.com.br/imovel/894601086/alugar/apartamento-2-quartos-santa-ifigenia-sao-paulo?from_route=%22house%20details%22&search_id=%229c5941646d2911ef9c4d8e0e165fee2f%22&search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%7D
valor_total_mensal: Não informado
Dados do imóvel não foram inseridos no CSV.
Dados colet